# Lab 10 - Web Mining
## Advait Deochakke
###     20BCE1143

In [8]:
import networkx as nx
import numpy as np

In [49]:

G = nx.DiGraph()

# Add the nodes
G.add_nodes_from(['A', 'B', 'C', 'D', 'E', 'F'])

# Add the edges
G.add_edges_from([('A', 'B'), ('A', 'D'), ('A', 'E'), ('A', 'F'),
                  ('B', 'A'), ('B', 'C'), ('B', 'D'),
                  ('C', 'B'), ('C', 'D'),
                  ('D', 'A'), ('D', 'B'), ('D', 'C'),
                  ('E', 'A'), ('E', 'F'),
                  ('F', 'A'), ('F', 'E')])
# it takes you about two hours to realize that working with a complete weighted, directed graph with
# base weight as zero to show no edge and weight as 1 to show edge
# is so much easier to work with than whatever i was tyring to do above

print(G)

DiGraph with 6 nodes and 16 edges


In [62]:
nx.pagerank(G, alpha=0.9, max_iter=100)

{'A': 0.24432008217628517,
 'B': 0.18404146267950455,
 'C': 0.12709167694914336,
 'D': 0.18404146267950455,
 'E': 0.1302526577577811,
 'F': 0.1302526577577811}

In [86]:
def calc_pagerank(G, alpha=0.85, max_iter=100, tol=1e-4, weighted=False):
    
    """
    This function calculates the weighted page rank scores for a given graph G using the Weighted Page Rank algorithm as described by Wenpu Xing et al.
    
    :param G: The input graph
    :type G: networkx.DiGraph
    :param alpha: The damping factor (default value is 0.85)
    :type alpha: float
    :param max_iter: The maximum number of iterations to perform (default value is 100)
    :type max_iter: int
    :param tol: The tolerance used to check for convergence (default value is 1e-4)
    :type tol: float
    :return: A dictionary containing the page rank scores for each node in the graph
    :rtype: dict
    
    :param weighted: an input to say if we use the WPR or normal version
    default false
    """
    
    n = len(G.nodes)
    node_list = list(G.nodes)

    # Initializing the page rank scores
    pageranks = {node: 1/n for node in node_list}
    
    # WPR algorithm
    if weighted:
        # print("entered weighted")
        for i in range(max_iter):
            # print("iteration", i)
            new_pageranks = {node: 0 for node in node_list}
            
            for j in range(n):
                sum_pr = 0
                
                reference_set = list(G.successors(node_list[j]))
                sum_inlinks = sum(G.in_degree(node) for node in reference_set)
                sum_outlinks = sum(G.out_degree(node) for node in reference_set)
                # print("sum inlinks is", sum_inlinks)
                # print("current node is", list(G.nodes)[j])
                for node in reference_set:
                    sum_pr += pageranks[node]*(G.in_degree(node)/sum_inlinks)*(G.out_degree(node)/sum_outlinks) 
                    # PR(u) * Win(v, u) * Wout(v, u)
                    
                new_pageranks[node_list[j]] = (1 - alpha) + alpha * sum_pr 
            
            # Checking for convergence - boilerplate
            converged = True
            for node in node_list:
                if abs(new_pageranks[node] - pageranks[node]) > tol:
                    converged = False
                    # print("the abs diff was greater than tol for ", node)
                    break
            if converged:
                return pageranks
            
            pageranks = new_pageranks
    
    else:
        # print("entered normal")
        for i in range(max_iter):
            # print("iteration", i)
            new_pageranks = {node: 0 for node in node_list}
            
            for j in range(n):
                sum_pr = 0
                
                reference_set = list(G.predecessors(node_list[j]))
                for node in reference_set:
                    sum_pr += pageranks[node]/(G.out_degree(node))
                    
                new_pageranks[node_list[j]] = (1 - alpha) + alpha * sum_pr 
            
            # Checking for convergence - boilerplate
            converged = True
            for node in node_list:
                if abs(new_pageranks[node] - pageranks[node]) > tol:
                    converged = False
                    # print("the abs diff was greater than tol for ", node)
                    break
            if converged:
                return pageranks
            
            pageranks = new_pageranks
    
    return pageranks


In [87]:
print("weighted page rank :")
wpr = calc_pagerank(G, weighted=True)
print(wpr)
print("normal page rank :")
npr = calc_pagerank(G, weighted=False)
npr = {key: value/len(list(G.nodes)) for key, value in npr.items()}
print(npr)

weighted page rank :
{'A': 0.1996139589685682, 'B': 0.21379225462935914, 'C': 0.24082939716707513, 'D': 0.21379225462935914, 'E': 0.2488770935071386, 'F': 0.2488770935071386}
normal page rank :
{'A': 0.2410755932364277, 'B': 0.18249893766024683, 'C': 0.1284085396216346, 'D': 0.18249893766024686, 'E': 0.13255832784388386, 'F': 0.13255832784388386}


In [90]:
print("page rank with networkx inbuilt :")
nxpr = nx.pagerank(G)
print(nxpr)

page rank with networkx inbuilt :
{'A': 0.24117597909181454, 'B': 0.18257382216198692, 'C': 0.12845786661086428, 'D': 0.1825738221619869, 'E': 0.13260925498667359, 'F': 0.13260925498667359}
